In [18]:
import psycopg2 as ps
import pandas as pd

In [47]:
conn = ps.connect("dbname=ecommerce_delivery_analysis user=postgres password=1234 host=localhost port=5432")

cur = conn.cursor()

In [ ]:
''' Loading Customer Data into Postgres'''

cur.execute('''
create table customer_info(
             customer_id TEXT PRIMARY KEY
             )
''')

conn.commit()

path = r'../dataset/dimensions/dim_customer_info.csv'

with open(path, 'r', encoding='utf-8') as f:
    next(f)  # skip header
    cur.copy_from(f, 'customer_info', sep=',', columns=('customer_id',))

conn.commit()


In [ ]:
''' Loading Platform Info into Postgres'''


cur.execute('''

create table platform_info (
           platform_id int primary key ,
           platform_name text unique,
           platform_code text unique
           )
''')

conn.commit()


path = r'../dataset/dimensions/dim_platform_info.csv'

with open(path,'r') as f:
    next(f)  # skip header
    cur.copy_from(
        f,
        'platform_info',
        sep=',',
        columns=('platform_id', 'platform_name', 'platform_code')
    )

conn.commit()

In [ ]:
''' Loading Product Categories into Postgres'''

cur.execute('''
create table product_categories(
           category_id int primary key,
           category_name text unique
           )
''')

conn.commit()

path = r'../dataset/dimensions/dim_product_categories.csv'

with open(path,'r') as f:
    next(f)
    cur.copy_from(f,'product_categories',sep=',', columns=('category_id','category_name'))

conn.commit()

In [ ]:
''' Loading Orders Data into Postgres'''

cur.execute('''

CREATE TABLE orders (
    order_id TEXT PRIMARY KEY NOT NULL,
    customer_id TEXT REFERENCES customer_info(customer_id),
    platform_id INT REFERENCES platform_info(platform_id),
    product_category_id INT REFERENCES product_categories(category_id),
    order_datetime TEXT,
    delivery_time_min INT,
    order_value_inr INT,
    delivery_delay TEXT,
    refund_request TEXT,
    service_rating INT,
    customer_feedback TEXT
)

''')

conn.commit()


path = r'../dataset/dimensions/dim_orders.csv'

with open(path, 'r') as f:
    cur.copy_expert("""
        COPY orders (
            order_id,
            customer_id,
            platform_id,
            product_category_id,
            order_datetime,
            delivery_time_min,
            order_value_inr,
            delivery_delay,
            refund_request,
            service_rating,
            customer_feedback
        )
        FROM STDIN WITH CSV HEADER DELIMITER ','
    """, f)

conn.commit()


In [ ]:
cur.execute('''

create table raw_data (
            
            order_id text primary key,
            customer_id text,
            platform text,
            order_date_time text,
            delivery_min int,
            product_category text,
            order_value int,
            customer_feedback text,
            service_rating int,
            delivery_delay text,
            refund_requested text   
            )

''')

conn.commit()


path = r'..\dataset\raw_data\ecommerce_delivery_analytics.csv'

with open(path, 'r') as f:
    cur.copy_expert("""
        COPY raw_data (
            order_id,
            customer_id,
            platform,
            order_date_time,
            delivery_min,
            product_category,
            order_value,
            customer_feedback,
            service_rating,
            delivery_delay,
            refund_requested
        )
        FROM STDIN WITH CSV HEADER DELIMITER ','
    """, f)

# Commit to save the data
conn.commit()


In [ ]:
''' Transformation is done directly in PostgreSQL you can find queries in SQL folder '''